### nuScenes to poses (runtime: ~ 3min)

In [3]:
import numpy as np
import math
import os
import json
from tqdm import tqdm
from typing import List, Tuple, Dict
from nuscenes.nuscenes import NuScenes

#### Set paths
<div class="alert alert-block alert-warning">
<b>ToDo:</b> 
<ol>
    <li> Set "datasets_root" to the root of your <b>dataset directory</b>
    <li> Set "vanishing_data_root" to your <b>working directory</b>
    <li> Set "coda_root" to your <b>working directory</b>
    <li> Select if <b>oxts</b> should be saved (runtime: ~ 7h)
</ol>
</div>

In [4]:
datasets_root = f'/disk/ml/datasets'
vanishing_data_root = f'/disk/vanishing_data/ju878'
coda_root = f'/disk/ml/own_datasets/CODA'

save_oxts = False

In [5]:
nuscenes_root = os.path.join(datasets_root, 'nuScenes')

prepared_poses = os.path.join(vanishing_data_root, 'prepared_poses')
oxts_folder = os.path.join(prepared_poses, 'nuscenes/oxts')
poses_folder = os.path.join(prepared_poses, 'nuscenes/poses')

json_tokens = os.path.join(coda_root, 'nuscenes_indices.json')

In [6]:
EARTH_RADIUS_METERS = 6.378137e6
REFERENCE_COORDINATES = {
    "boston-seaport": [42.336849169438615, -71.05785369873047],
    "singapore-onenorth": [1.2882100868743724, 103.78475189208984],
    "singapore-hollandvillage": [1.2993652317780957, 103.78217697143555],
    "singapore-queenstown": [1.2782562240223188, 103.76741409301758],
}

OXTS_LINE_LEN = 6

In [7]:
with open(json_tokens, 'r') as f:
    nuscenes_tokens = json.load(f)
    
if not os.path.exists(oxts_folder) and save_oxts:
        os.makedirs(oxts_folder)
if not os.path.exists(poses_folder):
        os.makedirs(poses_folder)

#### Load nuScenes

In [8]:
nusc = NuScenes(dataroot=nuscenes_root, version='v1.0-trainval', verbose=False)

#### Functions to create poses

Get poses

In [9]:
def get_poses(nusc: NuScenes, scene_token: str) -> List[dict]:
    pose_list = []
    scene_rec = nusc.get('scene', scene_token)
    sample_rec = nusc.get('sample', scene_rec['first_sample_token'])
    sd_rec = nusc.get('sample_data', sample_rec['data']['LIDAR_TOP'])
    
    
    ego_pose = nusc.get('ego_pose', sd_rec['token'])
    #if sd_rec['token'] in lidar_top_tokens:
    pose_list.append(ego_pose)

    while sd_rec['next'] != '':
        sd_rec = nusc.get('sample_data', sd_rec['next'])
        ego_pose = nusc.get('ego_pose', sd_rec['token'])
        #if sd_rec['token'] in lidar_top_tokens:
        pose_list.append(ego_pose)

    return pose_list

Get coordinates

In [10]:
def get_coordinate(ref_lat: float, ref_lon: float, bearing: float, dist: float) -> Tuple[float, float]:
    lat, lon = math.radians(ref_lat), math.radians(ref_lon)
    angular_distance = dist / EARTH_RADIUS_METERS
    
    target_lat = math.asin(
        math.sin(lat) * math.cos(angular_distance) + 
        math.cos(lat) * math.sin(angular_distance) * math.cos(bearing)
    )
    target_lon = lon + math.atan2(
        math.sin(bearing) * math.sin(angular_distance) * math.cos(lat),
        math.cos(angular_distance) - math.sin(lat) * math.sin(target_lat)
    )
    return math.degrees(target_lat), math.degrees(target_lon)

Get shortened oxts (only values needed for poses)

In [11]:
def derive_latlon(location: str, poses: List[Dict[str, float]], scene_name) -> List[Dict[str, float]]:
    assert location in REFERENCE_COORDINATES.keys(), \
        f'Error: The given location: {location}, has no available reference.'
    
    oxts_scene_folder = os.path.join(oxts_folder, scene_name)
    oxts = []
    reference_lat, reference_lon = REFERENCE_COORDINATES[location]
    
    if save_oxts:
        if not os.path.exists(oxts_scene_folder):
            os.makedirs(oxts_scene_folder)
    
    for p in poses:
        pose_token = p['token']
        
        x, y, z = p['translation']
        bearing = math.atan(x / y)
        distance = math.sqrt(x**2 + y**2)
        lat, lon = get_coordinate(reference_lat, reference_lon, bearing, distance)
        
        w, r1, r2, r3 = p['rotation']
        roll = math.atan2(2 * (w * r1 + r2 * r3), 1 - 2 * (math.pow(r1, 2) + math.pow(r2, 2)))
        pitch = math.asin(2 * (w * r2 - r3 * r1))
        yaw = math.atan2(2 * (w * r3 + r1 * r2), 1 - 2 * (math.pow(r2, 2) + math.pow(r3, 2)))
        
        oxts.append([lat, lon, z, roll, pitch, yaw])
        
        if save_oxts:
            with open(os.path.join(oxts_scene_folder, pose_token + '.txt'), 'w') as f:
                f.write(f'{lat} {lon} {z} {roll} {pitch} {yaw}')
    return oxts

Helper functions

In [12]:
def lat_lon_to_mercator(lat, lon, scale):
    ER = 6378137
    mx = scale * lon * math.pi * ER / 180
    my = scale * ER * math.log(math.tan((90 + lat) * math.pi / 360))
    return mx, my


def lat_to_scale(lat):
    return math.cos(lat * math.pi / 180.0)

Translate oxts to poses

In [13]:
def gps_imu_to_pose(gps_imu, scale):
    t = np.zeros(3)
    t[0], t[1] = lat_lon_to_mercator(gps_imu[0], gps_imu[1], scale)
    t[2] = gps_imu[2]  # altitude
    rx = gps_imu[3]  # roll
    ry = gps_imu[4]  # pitch
    rz = gps_imu[5]  # heading
    Rx = np.array([[1, 0, 0], [0, math.cos(rx), -math.sin(rx)], [0, math.sin(rx), math.cos(rx)]])
    Ry = np.array([[math.cos(ry), 0, math.sin(ry)], [0, 1, 0], [-math.sin(ry), 0, math.cos(ry)]])
    Rz = np.array([[math.cos(rz), -math.sin(rz), 0], [math.sin(rz), math.cos(rz), 0], [0, 0, 1]])
    R = Rz.dot(Ry).dot(Rx)
    T = np.identity(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T

def oxts_to_pose(oxts_info):
    # Compute scale from first lat value
    scale = lat_to_scale(oxts_info[0][0])
    Tr_0_inv = None
    poses = np.zeros((len(oxts_info), 12))
    for i, line in enumerate(oxts_info):
        T = gps_imu_to_pose(line, scale)
        # Normalize translation and rotation (start at 0/0/0)
        if Tr_0_inv is None:
            Tr_0_inv = np.linalg.inv(T)

        pose = Tr_0_inv.dot(T)
        poses[i] = pose[:3, :].reshape(12)
    return poses

#### Translate nuScenes_poses to KITTI_poses

In [ ]:
for scene in tqdm(nusc.scene):
    scene_name = scene['name']
    scene_token = scene['token']
    
    location = nusc.get('log', scene['log_token'])['location']
    
    nuscenes_poses = get_poses(nusc, scene_token)
    
    #oxts_data = load_oxts_data(scene_token, os.path.join(base_dir, scene_name))
    oxts = derive_latlon(location, nuscenes_poses, scene_name)
    
    poses = oxts_to_pose(oxts)
    np.savetxt(os.path.join(poses_folder, scene_name + '.txt'), poses)